In [3]:
import duckdb
con = duckdb.connect(':memory:')

In [5]:
sql = """
CREATE TABLE "Generation History" AS
    FROM '../power-plant-generation-history.csv'
"""
con.execute(sql)

Range Framing Window

In [ ]:
sql = """
SELECT "Plant", "Date",
    min("MWh") OVER seven AS "MWh 7-day Moving Minimum",
    avg("MWh") OVER seven AS "MWh 7-day Moving Average",
    max("MWh") OVER seven AS "MWh 7-day Moving Maximum",
    min("MWh") OVER three AS "MWh 3-day Moving Minimum",
    avg("MWh") OVER three AS "MWh 3-day Moving Average",
    max("MWh") OVER three AS "MWh 3-day Moving Maximum"
FROM "Generation History"
WINDOW
    seven AS (
        PARTITION BY "Plant"
        ORDER BY "Date" ASC
        RANGE BETWEEN INTERVAL 3 DAYS PRECEDING
                  AND INTERVAL 3 DAYS FOLLOWING),
    three AS (
        PARTITION BY "Plant"
        ORDER BY "Date" ASC
        RANGE BETWEEN INTERVAL 1 DAYS PRECEDING
        AND INTERVAL 1 DAYS FOLLOWING)
ORDER BY 1, 2
"""
con.sql(sql)

In [ ]:
sql = """
SELECT "Plant", "Date",
    min("MWh") OVER seven AS "MWh 7-day Moving Minimum",
    avg("MWh") OVER seven AS "MWh 7-day Moving Average",
    max("MWh") OVER seven AS "MWh 7-day Moving Maximum",
    min("MWh") OVER three AS "MWh 3-day Moving Minimum",
    avg("MWh") OVER three AS "MWh 3-day Moving Average",
    max("MWh") OVER three AS "MWh 3-day Moving Maximum"
FROM "Generation History"
WINDOW
    seven AS (
        PARTITION BY "Plant"
        ORDER BY "Date" ASC
        RANGE BETWEEN INTERVAL 6 DAYS PRECEDING AND CURRENT ROW
    ),
    three AS (
        PARTITION BY "Plant"
        ORDER BY "Date" ASC
        RANGE BETWEEN INTERVAL 2 DAYS PRECEDING AND CURRENT ROW
    )
ORDER BY 1, 2
"""
con.sql(sql)

QUANTILE_CONT is a window function that calculates a specific percentile value (continuous) within a group of rows defined by a window frame. 
This function is used to compute a value that divides the group of rows into two parts: the proportion of rows below the computed value is equal to or less than a specified fraction.

https://duckdb.org/docs/sql/functions/aggregates#quantile_contx-pos

QUANTILE_CONT(fraction) WITHIN GROUP (ORDER BY expression) OVER (window_frame)


In [11]:
sql = """
SELECT "Plant", "Date",
    min("MWh") OVER seven AS "MWh 7-day Moving Minimum",
    quantile_cont("MWh", [0.25, 0.5, 0.75]) OVER seven
        AS "MWh 7-day Moving IQR",
    max("MWh") OVER seven AS "MWh 7-day Moving Maximum",
FROM "Generation History"
WINDOW seven AS (
    PARTITION BY "Plant"
    ORDER BY "Date" ASC
    RANGE BETWEEN INTERVAL 3 DAYS PRECEDING
              AND INTERVAL 3 DAYS FOLLOWING)
ORDER BY 1, 2;
"""
con.sql(sql)

┌───────────┬────────────┬──────────────────────────┬─────────────────────────────────┬──────────────────────────┐
│   Plant   │    Date    │ MWh 7-day Moving Minimum │      MWh 7-day Moving IQR       │ MWh 7-day Moving Maximum │
│  varchar  │    date    │          int64           │            double[]             │          int64           │
├───────────┼────────────┼──────────────────────────┼─────────────────────────────────┼──────────────────────────┤
│ Boston    │ 2019-01-02 │                   469538 │ [497938.25, 517964.0, 537476.5] │                   564337 │
│ Boston    │ 2019-01-03 │                   469538 │ [474163.0, 507405.0, 528523.0]  │                   564337 │
│ Boston    │ 2019-01-04 │                   469538 │ [482425.5, 507309.0, 523243.5]  │                   564337 │
│ Boston    │ 2019-01-05 │                   469538 │ [490688.0, 507405.0, 546430.0]  │                   613040 │
│ Boston    │ 2019-01-06 │                   469538 │ [490688.0, 507405.0, 55555